In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle as pkl



import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

from IPython.display import display, HTML

import ipywidgets as widgets
from ipywidgets import Layout

from difflib import get_close_matches
from scipy.signal import savgol_filter

import h5py
from joblib import Parallel, delayed

import random
random.seed(22)

In [2]:
sys.path.append("../code/")
from extract_angles import get_bend_angles, get_tan_angles
from extract_curvatures import get_curv_savgol

In [3]:
%matplotlib widget

## Load skeleton files

In [4]:
skel_file_folder = "/share/data/temp/athira/tierpsy_skeleton_files_for_eigen_npy/"
#skel_file_folder = "/media/athira/Seagate Expansion Drive/Backup_Feb26_2020/athira_data/tierpsy_skeleton_files_for_eigen_npy/"

In [5]:
# From the saved skeleton arrays, pick the ones belonging to wild or control group (and exclude INVERTED for now)

skel_files_control = [os.path.join(root,name) for root,dirs,filenames in os.walk(skel_file_folder) 
                                              for name in filenames if ((name.endswith('.npy')) & 
                                                                       (name.split("_")[5] == "None") #& 
                                                                       #(name.split("_")[7] == "None") #& 
                                                                       # ('INVERTED' not in name) &
                                                                       #(name.split("_")[0] == "12n")
                                                                       )]
#print([os.path.basename(fname).split("_")[4] for fname in skel_files_control])
print(f"Number of skeleton files which belong to control group:{len(skel_files_control)}")

Number of skeleton files which belong to control group:95


In [6]:
# select randomly n skeleton files

#n_files = 22
#skel_files = random.sample(skel_files_control, n_files)
skel_files = skel_files_control

In [7]:
# Load all the skeletons as a list of 3d arrays

skel_arrays = [np.load(skel_file) for skel_file in skel_files]

In [8]:
def preprocess_skels(skel_arrays, neck_point = 12, head_as = "original", tail_end = -1):
    
    if head_as == "omit":
        skel_arrays = [skel_arr[:,neck_point-2:tail_end,:] for skel_arr in skel_arrays]
        
    elif head_as == "rigid":
        skel_arrays = [np.hstack([skel_arr[:,0,:].reshape(-1,1,2),
                                  skel_arr[:,neck_point-1:tail_end,:]]) for skel_arr in skel_arrays]
        
    else:
        skel_arrays = [skel_arr[:,:tail_end,:] for skel_arr in skel_arrays]
    
        
    return skel_arrays

## Calculate shape features

### Calculate tangent angles

In [9]:
# calculate tangent angles for multiple skeletons in parallel
def get_tan_list(skel_arrays, filt = True):
    
    if filt:
        skel_arrays = [savgol_filter(skel_arr, window_length=45, polyorder=2, mode='nearest', axis=1) 
                                                    for skel_arr in skel_arrays]
        
    tan_angles_list = Parallel(n_jobs=10, verbose = 5)(delayed(get_tan_angles)
                                                    (skel_arr, filt=True, uni_lat=True, center =True) 
                                                for skel_arr in skel_arrays)
    
    return tan_angles_list

### Calculate bending angles

In [10]:
# calculate bending angles for multiple skeletons in parallel
def get_bend_list(skel_arrays, filt=False):
    
    if filt:
        skel_arrays = [savgol_filter(skel_arr, window_length=15, polyorder=2, mode='nearest') 
                                                    for skel_arr in skel_arrays]
    
    bend_angles_list = Parallel(n_jobs=10, verbose = 5)(delayed(get_bend_angles)(skel_arr) 
                                                    for skel_arr in skel_arrays)
    return bend_angles_list

### Calculate curvatures 

In [11]:
def get_curv_list(skel_arrays):
    
    curv_list = Parallel(n_jobs= 10, verbose = 5)(delayed(get_curv_savgol)(skel_arr) 
                                                for skel_arr in skel_arrays)
    return curv_list

## Eigen ciona calculations

In [12]:
# Using curvatures

# Load all the skeletons as a list of 3d arrays
skel_arrays = [np.load(skel_file) for skel_file in skel_files]

tail_end = -1
neck_point = 12
skel_arrays_orig = preprocess_skels(skel_arrays, neck_point = 12, head_as = "original", tail_end=tail_end)


#curv_list_orig = get_curv_list(skel_arrays_orig)
curv_list_orig = get_curv_list(skel_arrays_orig)


# if rigid:
#     curv_list_orig = [np.hstack([np.zeros_like(curv_mat[:,0:neck_point-1]),curv_mat[:,neck_point-1:tail_end]]) 
#                                           for curv_mat in curv_list_orig]

curv_matrix_orig = np.vstack(curv_list_orig)


cov_matrix_orig = np.cov(curv_matrix_orig, rowvar= False)



# Compute the eigen values and vectors of the covariance matrix 
eigen_vals_orig, eigen_vecs_orig = np.linalg.eig(cov_matrix_orig)


# sort the eigen values and vectors
idx = eigen_vals_orig.argsort()[::-1]   
eigen_vals_orig = eigen_vals_orig[idx]
eigen_vecs_orig = eigen_vecs_orig[:,idx]

# calculate the fractional variance
eigen_vals_orig_ = eigen_vals_orig
eigen_vals_orig = np.cumsum(eigen_vals_orig)/np.sum(eigen_vals_orig) 


thresh = 0.97
# Calculate how many eigen values are needed to explain 'thresh' fraction of the variance
n_modes_orig = np.sum((eigen_vals_orig < thresh) == True)

print(n_modes_orig)


eig_dict_orig = {'name': "original",'cov_mat': cov_matrix_orig, 'eig_vals': eigen_vals_orig, 
                 'eig_vals_':eigen_vals_orig_,'eig_vecs': eigen_vecs_orig, 'n_modes': n_modes_orig}

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    1.1s
[Parallel(n_jobs=10)]: Done  95 out of  95 | elapsed:    1.3s finished


6


## Plotting functions

In [13]:
def plot_cov_matrix( ax, cov_mat):
        
    cov_plot = ax.imshow(cov_mat, cmap='jet')
    plt.colorbar(cov_plot, ax=ax)
    ax.set_xlabel('covariance')

    return cov_plot


def plot_eig_vals(ax, eig_vals):

    eig_val_plot = ax.plot(eig_vals,'.')
    ax.set_xlabel('K mode')
    ax.set_ylabel('fractional variance explained')
    ax.set_ylim([0,1])

    return eig_val_plot


def plot_eig_vecs(ax, eig_vec, n_eig):

    eig_vec_plot = ax.set_ylabel(f"{n_eig}_eigen_vector")
    ax.plot(eig_vec)

    return eig_vec_plot

In [14]:
fig,axes = plt.subplots(min(n_modes_orig+2,8),1,figsize=(5,20))


axes[0].set_title(eig_dict_orig['name'])
plot_cov_matrix(axes[0], eig_dict_orig['cov_mat'])
plot_eig_vals(axes[1], eig_dict_orig['eig_vals'])

for n_vec in range(min(eig_dict_orig['n_modes'], 6)):

    plot_eig_vecs(axes[n_vec+2], eig_dict_orig['eig_vecs'][:,n_vec], n_vec+1)


fig.tight_layout(pad=3.0)
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
np.save('eigen_dict_all.npy',eig_dict_orig)

## Eigen reconstruction

### Example

In [16]:
slider_file = widgets.IntSlider(value= 0,
                                        min=0,
                                        max= len(skel_arrays)-1,
                                        step=1,
                                        description='File:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')

slider_frame = widgets.IntSlider(value= 0,
                                        min=0,
                                        max= 5000,
                                        step=1,
                                        description='Frames:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')


    
@widgets.interact
def plot_sample_recon(ind_file = slider_file, ind_frame = slider_frame):
    
    slider_frame.max = len(skel_arrays[ind_file])-1
    
    sample_curv = curv_list_orig[ind_file][ind_frame,:]
    
    coeff_list = []
    for k in range(eig_dict_orig['n_modes']):

        coeff_list.append(np.dot(eig_dict_orig['eig_vecs'][:,k],sample_curv))
    
    scaled_eig_vecs = eig_dict_orig['eig_vecs'][:,0:eig_dict_orig['n_modes']] * coeff_list
    recon_curv = np.sum(scaled_eig_vecs,axis = 1)


    fig, axes = plt.subplots(1,1, figsize =(10,5))
    axes.plot(sample_curv , label = 'curv')
    axes.plot(recon_curv, label= 'recon curv')
    
    axes.legend()

interactive(children=(IntSlider(value=0, continuous_update=False, description='File:', max=94), IntSlider(valu…

### Eigen Coefficients Time Series

- Reference : [Drosophila paper](https://royalsocietypublishing.org/doi/10.1098/rsif.2015.0899)

In [17]:
eig_vec_matrix = np.array(eig_dict_orig['eig_vecs'][:,0:eig_dict_orig['n_modes']])
eig_vec_matrix.shape

(48, 6)

In [22]:
for ind, file in enumerate(skel_files_control):
    fname = os.path.basename(file)
    
    if fname == "12n_20180823_150140_1_5m0s_None_None_None_skeleton.npy":
        print(ind)

62


In [23]:
# Choose one experiment 

ind_file = 62

curv_list_orig[ind_file].shape

(9001, 48)

#### Trace back tierpsy features to compare with ects!

In [24]:
skel_fname = os.path.basename(skel_files_control[ind_file])
feature_files_folder = "/share/data/temp/daniel/20200211_tierpsy_features/"
(root, dirs, filenames) = next(os.walk(feature_files_folder))
skels_selected = [fname for fname in filenames if skel_fname[:20] in fname]
    
print(skels_selected)
skel_file_path = os.path.join(feature_files_folder,skels_selected[0])
skel_feats = np.load(skel_file_path, allow_pickle=True)
display(skel_feats)

['12n_20180823_150140_1_5m0s_None_None_None_features_JDA.pickle']


,skel_lenght,area,frame_number,width_head,width_tail,width_tail_tip,quirkiness,curvature_head,curvature_tail_neck,curvature_tail_middle,...,relative_to_tail_middle1_radial_velocity_tail_tip,relative_to_tail_middle1_angular_velocity_tail_tip,speed_tail_middle,angular_velocity_tail_middle,speed_head_tip,angular_velocity_head_tip,speed_tail_tip,angular_velocity_tail_tip,speed_all,angular_velocity_all
0,124.426407,508.941559,0,9.534327,3.605551,2.828427,0.998633,-0.014143,-1.279202e-02,-7.818981e-03,...,-5.310745,-1.178545,11.790259,0.029293,5.459580,-0.024762,4.659012,-0.237079,3.745293,0.000000
1,122.669044,508.550751,1,9.769024,3.605551,2.828427,0.998540,-0.010608,-1.709121e-02,-1.247935e-03,...,-10.769576,0.852374,5.236989,0.259180,4.229090,-0.392178,6.805718,0.635219,4.584215,0.000000
2,125.597977,505.697937,2,9.721663,3.605551,2.595768,0.998520,-0.007847,-1.070164e-03,-2.657407e-03,...,6.141014,-0.404545,22.157988,0.058440,2.499563,1.061808,26.762848,-0.026793,9.957694,0.099124
3,126.012192,506.919403,3,9.461916,3.869978,2.236068,0.998445,-0.008854,-1.060749e-02,1.588150e-07,...,-5.993042,0.468761,24.067905,-0.058440,2.480309,-0.971503,26.923241,0.026793,13.453935,-0.099124
4,125.597977,507.774780,4,9.757098,3.605551,2.828427,0.998695,-0.011470,-7.688750e-03,3.799246e-03,...,-3.115539,-0.447260,20.490078,0.058665,0.104617,0.165943,15.962188,-2.149365,14.229018,-0.254062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,126.497475,525.303650,8996,9.390701,3.605551,2.236068,0.998723,0.001539,1.853257e-02,1.639204e-02,...,1.756782,-2.135074,64.125259,-0.255557,52.263550,-1.364028,54.567066,0.954638,55.384804,0.000000
8997,126.840622,528.864929,8997,9.813562,3.764757,2.828427,0.998920,-0.008024,6.071094e-03,2.718119e-02,...,-9.475593,0.005475,38.930943,-0.173006,27.280182,-0.102428,30.292732,0.060182,29.778297,0.000000
8998,125.733543,524.254395,8998,9.716990,3.605551,2.828427,0.998979,-0.011817,9.704877e-03,3.071829e-03,...,0.930176,-0.837218,9.146766,-0.396320,19.909374,0.201334,10.347622,-1.084456,10.905099,-0.248741
8999,126.490906,528.483521,8999,9.391096,3.642266,2.828427,0.998840,-0.024489,-2.674310e-07,1.923335e-02,...,40.198513,1.276871,36.364044,0.920151,14.822331,-1.043837,11.201249,2.060359,22.751928,0.248741


In [25]:
# duration_sec = int(os.path.basename(skel_files_control[ind_file]).split("_")[4].split('m')[0]) * 60
# frame_rate = len(curv_list_orig[ind_file])/duration_sec
# frame_rate

In [26]:
ects = np.dot(curv_list_orig[ind_file],eig_vec_matrix)
ects.shape

(9001, 6)

In [27]:
## To check if the matrix calculations makes sense, 

slider_frame = widgets.IntSlider(value= 0,
                                        min=0,
                                        max= len(curv_list_orig[ind_file])-1,
                                        step=1,
                                        description='Frames:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')


    
@widgets.interact
def plot_recon_mat(ind_frame = slider_frame):
    fig, axes = plt.subplots(1,1, figsize =(10,5))
    recon_curv = np.sum(ects[ind_frame]*eig_vec_matrix, axis=1)
    axes.plot(curv_list_orig[ind_file][ind_frame,:] , label = 'curv')
    axes.plot(recon_curv, label= 'recon curv')
    axes.legend()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Frames:', max=9000), Output()),…

In [28]:

# slider_frame_range = widgets.IntRangeSlider(value=[0, 800],
#                                             min=0,
#                                             max=len(ects),
#                                             step=1,
#                                             description='Test:',
#                                             disabled=False,
#                                             continuous_update=False,
#                                             orientation='horizontal',
#                                             readout=True,
#                                             readout_format='d',layout=Layout(width='900px'))
# @widgets.interact
# def plot_ects(inds_frame = slider_frame_range):
#     ind_start, ind_end = inds_frame
#     fig,axes = plt.subplots(2,1,figsize=(20,8))
#     axes[0].plot(ects[ind_start:ind_end,:])
#     axes[1].plot(skel_feats['speed_all'][ind_start:ind_end],label='speed')
#     plt.legend()

### Density plots for ects

In [29]:
from scipy.stats import gaussian_kde
  
fig, axes = plt.subplots(1,1,figsize=(10,5))

for i in range(eig_dict_orig['n_modes']):
        sns.set_style('whitegrid')
        sns.kdeplot(ects[:,i], ax=axes, label=f"coeff_{i}")
        axes.set_xlabel('eigen coefficient value')
        axes.set_ylabel('density')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
plt.close('all')
fig, axes = plt.subplots(1,1,figsize=(5,5))

sns.kdeplot(ects[:,0],ects[:,2], ax=axes, cmap='jet', shade=True)
axes.set_xlabel('a1')
axes.set_ylabel('a2')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'a2')

### smoothening ECTS
- using weighted running average filter

In [31]:

def get_weights(window_length):
    weights = np.ones(window_length)
    for i in range(len(weights)):
        weights[i] = 1/ (1 + (np.abs(i-((len(weights) -1)/2))))
           
    return weights
    


In [32]:
def weighted_average_filter(x, window_length = 15):
    
    pad_len = window_length // 2
    x_padded = np.pad(x,((pad_len, pad_len),(0,0)), mode='edge')
    print(x.shape, x_padded.shape)
    
    x_filt = np.zeros_like(x)
    for i in range(len(x_filt)):
        
        x_filt[i,:] = np.average(x_padded[i:(i+2*pad_len)+1,:], 
                                           weights = get_weights(window_length),axis=0)
    
    return x_filt

wl = 5

ects_filt = weighted_average_filter(ects, wl)

(9001, 6) (9005, 6)


In [ ]:
#np.savetxt('./temp_ects.txt',ects_filt,delimiter=',')

In [39]:
slider_frame2 = widgets.IntRangeSlider(value=[0, 5000],
                                            min =0, 
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))



@widgets.interact
def plot_bscore(inds_frame = slider_frame2):
    ind_start, ind_end = inds_frame
    x_range = np.arange(ind_start, ind_end)
    fig,axes = plt.subplots(eig_dict_orig['n_modes']+2,1,figsize=(8,8))
    
    ects_lines = axes[0].plot(x_range,ects_filt[ind_start:ind_end,:])
    ects_labels = [f"coeff_{ind}" for ind in range(eig_dict_orig['n_modes'])]
    axes[0].legend(ects_lines,ects_labels)
    
    for i in range(eig_dict_orig['n_modes']):
        axes[i+2].plot(x_range,ects[ind_start:ind_end,i], label='original')
        axes[i+2].plot(x_range,ects_filt[ind_start:ind_end,i], label='filtered')
        axes[i+2].legend()
        
    axes[1].plot(skel_feats['speed_all'][ind_start:ind_end],label='speed')
    axes[1].legend()
    plt.xlabel('frames')
    plt.show()

interactive(children=(IntRangeSlider(value=(0, 5000), continuous_update=False, description='Test:', layout=Lay…

In [33]:
speed_bp_params = ['speed_head', 'speed_tail_neck','speed_tail_middle','speed_tail_middle1', 'speed_tail_tip']
skel_feats_speed = skel_feats.filter(items=speed_bp_params)

In [34]:
df_ects = pd.DataFrame(ects)

In [35]:
corr_ects = df_ects.corr()
corr_speed = skel_feats_speed.corr()

fig, axes = plt.subplots(1,2,figsize=(6,3))

sns.heatmap(corr_ects, ax=axes[0])
sns.heatmap(corr_speed, ax = axes[1])

plt.tight_layout(w_pad=3.0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
df_ects_filt = pd.DataFrame(ects_filt, columns=[f"a{i}" for i in range(ects_filt.shape[1])])

In [ ]:
plt.close('all')


slider_frame = widgets.IntRangeSlider(value=[0, 1000],
                                            min=0,
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))


@widgets.interact
def plot_trajectories(frames = slider_frame):
    frame_start, frame_end = frames
    
    coeff_scatter_plot = sns.PairGrid(df_ects_filt[frame_start:frame_end],  
                                      x_vars=list(df_ects_filt.columns), 
                                      y_vars=list(df_ects_filt.columns))
    coeff_scatter_plot.fig.set_size_inches(8,8)
    coeff_scatter_plot.map_lower(sns.kdeplot, shade=True, cmap='jet')
    coeff_scatter_plot.map_upper(sns.scatterplot,s=5)
    coeff_scatter_plot.map_diag(sns.kdeplot, color=sns.color_palette()[2])

In [ ]:

slider_frame1 = widgets.IntRangeSlider(value=[0, 1000],
                                            min=0,
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))

@widgets.interact
def plot_trajectories(frames = slider_frame1):
    frame_start, frame_end = frames
    plt.figure(figsize=(5,5))
    ax = plt.axes(projection='3d')
    ax.scatter3D(ects_filt[frame_start:frame_end,0],ects_filt[frame_start:frame_end,1],
            ects_filt[frame_start:frame_end,2]
                 #,c = cm.autumn(ects_filt[frame_start:frame_end,3]))
                 ,c = np.arange(0,len(ects)-1)[frame_start:frame_end], cmap='jet')
    
    


In [77]:
body_score = np.average(ects_filt, weights=(eig_dict_orig['eig_vals_'][0:eig_dict_orig['n_modes']]), axis=1)

In [78]:
slider_frame_range = widgets.IntRangeSlider(value=[0, 5000],
                                            min=0,
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))
@widgets.interact
def plot_bscore(inds_frame = slider_frame_range):
    ind_start, ind_end = inds_frame
    fig,axes = plt.subplots(1,1,figsize=(15,3))
    axes.plot(body_score[ind_start:ind_end])
    #axes.plot(body_score_filt[ind_start:ind_end])

interactive(children=(IntRangeSlider(value=(0, 5000), continuous_update=False, description='Test:', layout=Lay…

### Segmentation 

In [57]:
local_minima = np.r_[True, body_score[1:] < body_score[:-1]] & np.r_[body_score[:-1] < body_score[1:], True]

In [58]:
slider_frame_range = widgets.IntRangeSlider(value=[0, 100],
                                            min=0,
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))
@widgets.interact
def plot_bscore(inds_frame = slider_frame_range):
    ind_start, ind_end = inds_frame
    fig,axes = plt.subplots(1,1,figsize=(20,8))
    
    local_minima_coords = [i for i, x in enumerate(local_minima[ind_start:ind_end]) if x]
    for xc in local_minima_coords:
        axes.axvline(x=xc,color='k', linestyle='--')
    
    axes.plot(body_score[ind_start:ind_end])

interactive(children=(IntRangeSlider(value=(0, 100), continuous_update=False, description='Test:', layout=Layo…

#### Segmentation and clustering using HMM

In [59]:
from hmmlearn import hmm

In [60]:
ects_filt[:,0].shape

(9001,)

In [71]:
X_train = body_score.reshape(-1,1)

model = hmm.GaussianHMM(n_components=3, covariance_type='full',n_iter=200, verbose=True)

model.fit(X_train)

         1       22728.3148             +nan
         2       40684.3865      +17956.0716
         3       40684.3866          +0.0001


GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
            min_covar=0.001, n_components=3, n_iter=200, params='stmc',
            random_state=None, startprob_prior=1.0, tol=0.01,
            transmat_prior=1.0, verbose=True)

In [72]:
if model.monitor_.converged:
    print("Model converged")

Model converged


In [73]:
clusters_train = model.predict(X_train)

In [74]:
plt.close('all')
slider_frame_range = widgets.IntRangeSlider(value=[0, 5000],
                                            min=0,
                                            max=len(ects)-1,
                                            step=1,
                                            description='Test:',
                                            disabled=False,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='d',layout=Layout(width='900px'))
@widgets.interact
def plot_bscore_hmm(inds_frame = slider_frame_range):
    ind_start, ind_end = inds_frame
    fig,axes = plt.subplots(1,1,figsize=(15,3))
    x_range = range(ind_start,ind_end)
    c_pal = sns.color_palette("hls", 8)
    colors = [ c_pal[clus] for clus in clusters_train[ind_start:ind_end]]
    axes.scatter(x_range, body_score[ind_start:ind_end], c= colors, s=2)
    #axes.plot(body_score_filt[ind_start:ind_end])

interactive(children=(IntRangeSlider(value=(0, 5000), continuous_update=False, description='Test:', layout=Lay…